In [19]:
import json
from urllib import request, error
import pandas as pd
from itertools import product

# Build OD

Data can be downloaded from https://drive.google.com/drive/folders/1GOhMd0UbGnod2Kgtyw5k8_3KYFlEBHT0?usp=sharing

In [20]:
# Read OD file
file = pd.read_csv('Data/accessibility/rework/OD_54km.csv', encoding='gbk')

In [21]:
# Check
file.columns

Index(['Index', 'IN_FID', 'NEAR_FID', 'NEAR_DIST', 'NEAR_RANK', '序号', '机构名',
       '机构地', '行政区', '机构类', '机构级', '经营性', '床位数', '牙椅数', 'wgs_lng', 'wgs_lat',
       'POP', 'lng_84', 'lon_84', 'gcj_lng_D', 'gcj_lat_D', 'gcj_lng_O',
       'gcj_lat_O'],
      dtype='object')

In [22]:
## Reduced latitude and longitude decimal places to 6 (API compliant)
file['gcj_lng_D_6'] =  file['gcj_lng_D'].apply(lambda x: float(f"{x:.6f}"))
file['gcj_lat_D_6'] =  file['gcj_lat_D'].apply(lambda x: float(f"{x:.6f}"))
file['gcj_lng_O_6'] =  file['gcj_lng_O'].apply(lambda x: float(f"{x:.6f}"))
file['gcj_lat_O_6'] =  file['gcj_lat_O'].apply(lambda x: float(f"{x:.6f}"))
## Connecting latitude and longitude
file['gcj_D'] = file.apply(lambda row: f"{row['gcj_lng_D_6']},{row['gcj_lat_D_6']}", axis=1)
file['gcj_O'] = file.apply(lambda row: f"{row['gcj_lng_O_6']},{row['gcj_lat_O_6']}", axis=1)
file['gcj_OD'] = file.apply(lambda row: f"{row['gcj_O']};{row['gcj_D']}", axis=1)

In [23]:
# Double check
len(file)

3429

In [24]:
file.head()

,Index,IN_FID,NEAR_FID,NEAR_DIST,NEAR_RANK,序号,机构名,机构地,行政区,机构类,...,gcj_lat_D,gcj_lng_O,gcj_lat_O,gcj_lng_D_6,gcj_lat_D_6,gcj_lng_O_6,gcj_lat_O_6,gcj_D,gcj_O,gcj_OD
0,0,0,410,356.067221,1,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,...,23.131318,113.254681,23.130864,113.258122,23.131318,113.254681,23.130864,"113.258122,23.131318","113.254681,23.130864","113.254681,23.130864;113.258122,23.131318"
1,106,0,629,12712.344540,107,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,...,23.131318,113.217453,23.239799,113.258122,23.131318,113.217453,23.239799,"113.258122,23.131318","113.217453,23.239799","113.217453,23.239799;113.258122,23.131318"
2,517,0,401,31715.007350,518,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,...,23.131318,113.566411,23.107887,113.258122,23.131318,113.566411,23.107887,"113.258122,23.131318","113.566411,23.107887","113.566411,23.107887;113.258122,23.131318"
3,572,0,402,33711.050870,573,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,...,23.131318,113.585891,23.107545,113.258122,23.131318,113.585891,23.107545,"113.258122,23.131318","113.585891,23.107545","113.585891,23.107545;113.258122,23.131318"
4,732,0,609,38939.503170,733,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,...,23.131318,113.627019,23.215234,113.258122,23.131318,113.627019,23.215234,"113.258122,23.131318","113.627019,23.215234","113.627019,23.215234;113.258122,23.131318"


# Call the API to calculate the time cost

In [25]:
# Prepare output file
output = open('Data/accessibility/rework/OD_Cost.txt', 'wb')

In [26]:
# Check
#for i in OD[:5]:
    #origin, destination = i.split(';')
    #print(url)

The next step may takes several hours.

In [27]:
for index, i in enumerate(file['gcj_OD']):
    # paras
    origin, destination = i.split(';')
    key = '551f5e5fadec9095b3fbd3fe20de292f'
    strategy = 5  # 0：Recomended Mode（Bus+Subway）；5：No-Subway Mode
    url = f"https://restapi.amap.com/v5/direction/transit/integrated?key={key}&origin={origin}&destination={destination}&city1=020&city2=020&strategy={strategy}&show_fields=cost"

    try:
        html = request.urlopen(url, timeout=30).read()
        js = json.loads(html)

        d = js.get('route', {}).get('distance', '未知距离') # distance
        transits = js.get('route', {}).get('transits', [])

        if transits:
            t = transits[0]['cost']['duration'] # travel time
            c = transits[0]['cost']['transit_fee'] #travel fare
        else:
            t = 0
            c = 0

        out = f"{index},{d},{t},{c}\n"
        output.write(out.encode('utf-8'))

    except Exception as e:
        print(repr(e))
        output.write(f"{index},{url}".encode('utf-8'))

output.close()

URLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)'))
RemoteDisconnected('Remote end closed connection without response')
URLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)'))
RemoteDisconnected('Remote end closed connection without response')
RemoteDisconnected('Remote end closed connection without response')
RemoteDisconnected('Remote end closed connection without response')
RemoteDisconnected('Remote end closed connection without response')
RemoteDisconnected('Remote end closed connection without response')


------------------------------

The last cell is referenced from https://www.bilibili.com/video/BV1Tu411o7HH/?spm_id_from=333.880.my_history.page.click&vd_source=5b3fc5cf8a7a6f65932dc659545c9c2a